In [1]:
#!/usr/bin/env python
# coding: utf-8

import pandas as pd

import dataSrc
import utilities

# Retrieve symbols and financials

sp500_tickers = dataSrc.sp500_symbols()
dow_tickers = dataSrc.dow_symbols()

# sp500_financials = dataSrc.financials(sp500_tickers)

df = dataSrc.financials(dow_tickers)

# df = pd.read_csv('./R/data/dow_financials.csv')
# df = df.drop(['Unnamed: 0'], axis=1)
# df = df.dropna()

conditions = ((df["Revenue_Growth"] > 0)
              & (df["GPMargin"] > 0)
              & (df["EPS_Growth"] > 0)
              & (df["ROE"] > 0)
              & (df["DPS_Growth"] > 0)
              & (df["DivYield"] > 0))
df = df[conditions]

# Compute historical momentum
# Average momentum of prev 6M, 12M and 24M

mom_list = []
for symbol in df["symbol"]:
    print('MOMENTUM:'+ symbol)
    m12_momentum = utilities.calculate_hist_momentum(symbol, 252)
    m24_momentum = utilities.calculate_hist_momentum(symbol, 504)
    m36_momentum = utilities.calculate_hist_momentum(symbol, 756)
    avg_momentum = (m12_momentum + m24_momentum + m36_momentum) / 3
    mom_list.append(avg_momentum)

df["momentum"] = mom_list

df["mom_rank"] = df["momentum"].rank()
df = df.sort_values(by=["mom_rank"], ascending=False)
# numRows = df.shape[0]
# numCols = df.shape[1]
# top10 = df.copy()
df = df[df["momentum"] > 0]

print(df)

watchlist = df[["symbol", "name"]].copy()

# Update current prices, 52W High and Discount %

currentPrices = []
highs = []
discounts = []

for symbol in watchlist["symbol"]:
    print('WATCHLIST:', symbol)
    currentPrice = utilities.get_current_price(symbol)
    high = utilities.calculate_prev_max_high(symbol, 252)
    discount_pct = (currentPrice - high) / high

    currentPrices.append(currentPrice)
    highs.append(high)
    discounts.append(discount_pct)

watchlist["CurrentPrice"] = currentPrices
watchlist["52W_High"] = highs
watchlist["Discount%"] = discounts

watchlist = watchlist.sort_values(by="Discount%")

watchlist

CRM: 1/30
WBA: 2/30
V: 3/30
NKE: 4/30
UNH: 5/30
TRV: 6/30
VZ: 7/30
INTC: 8/30
WMT: 9/30
JNJ: 10/30
DIS: 11/30
MCD: 12/30
JPM: 13/30
CAT: 14/30
BA: 15/30
AMGN: 16/30
DOW: 17/30
AAPL: 18/30
GS: 19/30
CSCO: 20/30
MSFT: 21/30
HD: 22/30
PG: 23/30
MRK: 24/30
IBM: 25/30
HON: 26/30
KO: 27/30
CVX: 28/30
AXP: 29/30
MMM: 30/30
WBA
NKE
TRV
VZ
INTC
JNJ
CSCO
MSFT
HD
CVX
MSFT
NKE
HD
JNJ
CSCO
TRV
INTC


,symbol,name,CurrentPrice,52W_High,Discount%
7,INTC,Intel Corporation,52.01,68.489998,-0.240619
21,HD,"The Home Depot, Inc.",329.24,345.690002,-0.047586
3,NKE,"NIKE, Inc.",167.79,174.380000,-0.037791
5,TRV,"The Travelers Companies, Inc.",160.84,162.710007,-0.011493
20,MSFT,Microsoft Corporation,304.36,305.840000,-0.004839
9,JNJ,Johnson & Johnson,179.44,179.920000,-0.002668
19,CSCO,"Cisco Systems, Inc.",58.22,58.340000,-0.002057


In [2]:
#!jupyter nbconvert --to script --no-prompt analysis.ipynb